# MoE 混合专家系统

**大模型的 MoE（Mixture of Experts，混合专家系统）** 是一种用于提升模型性能的架构设计，核心思想是将复杂的任务拆解为多个子任务，并由不同的“专家”（子模型）分别处理，最终通过动态组合各专家的结果得到输出。MoE从提出发展至今，已经有了非常多的变种，下面是一个典型的 MoE 架构示意图：

![MoE Struct](img/MoE-Struct.svg)

## MoE 在 Transformer 中的应用

在 Transformer 模型中，前馈神经网络（FeedForward Neural Network, FNN）是其核心组件之一，用于对注意力层输出的特征进行非线性变换，增强模型对特征的提取和表示能力。然而，随着模型规模的扩大和任务复杂性的增加，传统的FNN在处理复杂任务和大规模数据时遇到了局限性。为了克服这些局限性，混合专家模型（MoE）被引入到 Transformer 架构中。

![Experts FFNN](img/Experts-FNN.svg)

如上图所示，MoE 模型通过**将 Transformer 中的 FNN 层替换为 MoE 层** ，引入了多个专家网络 Experts 和路由（门控）网络 Router（Gate Network）。**每个专家网络通常是一个独立的 FNN** ，但可以针对不同类型的输入数据进行专门的训练，从而发展出独特的特征处理能力。路由（门控）网络则根据输入数据的特点，将数据动态地路由到最合适的专家网络进行处理。这种架构不仅保留了Transformer的全局建模能力，还通过MoE的稀疏计算特性，提高了模型的灵活性、适应性和表示能力。

## 稠密模型与专家模型的对比

采用 FNN 的 Transformer 模型被称为稠密模型，是因为在这些模型中，每个 FNN 层的所有参数在每次前向传播时都会参与计算，参数连接是密集的。而采用 MoE 的模型被称为稀疏模型，是因为在 MoE 架构中，对于每个输入的Token，只有部分专家被激活，其余专家不参与计算，参数连接是稀疏的。这种稀疏性使得MoE模型在保持较大参数量的同时，能够减少计算量，提高计算效率和资源利用。以下是二者详细对比表：

| 对比维度  | 稠密模型（Dense）               | 专家模型（MoE）                       |
|:------|:--------------------------|:--------------------------------|
| 模型结构  | 所有参数对每个输入完全激活，每个神经元都参与计算。 | 只有部分专家（子网络）对每个输入进行计算，由路由（门控）网络动态选择。 |
| 计算效率  | 计算开销随模型规模线性增长，适用于中小规模模型。  | 通过稀疏激活显著降低计算量，支持更大规模的模型。        |
| 模型性能  | 在简单任务中表现稳定，但在复杂任务中效率较低。   | 能够根据不同任务需求动态调整专家，适应性和灵活性强。      |
| 扩展性   | 随着模型规模增大，内存占用和计算量增加。      | 可以在保持计算成本相对固定的情况下扩展模型规模。        |
| 适用场景  | 适用于对实时性要求高、推理延迟敏感的场景。     | 适合大规模预训练和多任务学习场景。               |
| 实现复杂度 | 实现简单，训练过程相对稳定。            | 实现复杂，需要额外设计路由机制。                |

## 实现目标

本章的目标是实现一个经典的 MoE 架构网络，我们需要关注其两个关键的结构：

- **Experts 专家网络：** 专家网络是模型中的独立子网络，每个专家专注于处理 **特定的输入子空间或特定任务** （注意这里的专家并非指擅长某一项专业的专家，如化学专家、医学专家等，而是在输入子空间中处理特定任务的专家）。
- **Router  路由网络：** 路由网络也称门控网络（Gate Network），负责根据输入数据的特征，动态地决定哪些 Expert 应该被激活来处理当前的输入。

推荐观看：[BiliBili-【大模型算法】MoE 架构](https://www.bilibili.com/video/BV1Gj9ZYdE4N)

---

## 准备工作

---

引入必要的库和定义必要的常量

In [ ]:
import torch
import torch.nn.functional as F
from numpy import dtype
from torch import nn

## Experts 专家网络

---

专家网络本质上就是一个更小的前馈神经网络，所以其实现方式与前馈神经网络是一致的。

In [ ]:
class Expert(nn.Module):
    def __init__(self, dim, dim_hidden):
        super().__init__()
        self.w1 = nn.Linear(dim, dim_hidden, bias=False)
        self.w2 = nn.Linear(dim_hidden, dim, bias=False)
        self.w3 = nn.Linear(dim, dim_hidden, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

## Router 路由网络

---

路由的结构由一层前馈神经网络和一个 `Softmax` 模块构成，如下图所示：

![MoE Router](img/MoE-Router.svg)

### 1. 路由的负载均衡

在 MoE中，负载均衡是指在多个专家模型之间合理分配任务或计算资源，以确保系统的整体性能和效率最大化。MoE 架构通常由多个专家模型组成，每个专家模型负责处理特定类型的任务或数据。负载均衡的核心目标是根据任务的复杂度、资源需求以及专家模型的处理能力，动态地将任务分配给最合适的专家，从而避免某些专家过载而其他专家闲置的情况。通过负载均衡，MoE 系统能够更高效地利用计算资源，减少任务处理的延迟，并提高整体的吞吐量。

#### 辅助损失函数

在MoE的负载均衡中，辅助损失函数（Auxiliary Loss Function）是一种用于平衡各个专家模型工作负载的机制。其主要目的是确保在模型训练过程中，各个专家模型能够均匀地分配计算资源，避免某些专家过载而其他专家闲置的情况。一种常见的实现是通过 **变异系数（Coefficient of Variation, CV）** 来构造辅助损失函数，其中变异系数公式如下：

$$
CV = \frac{\sigma}{\mu}
$$

- $\mu$ 指专家平均路由概率的均值
- $\sigma$ 指所有专家平均路由概率的标准差

辅助损失函数公式如下：

$$
L_{aux} = \lambda \cdot CV^2 = \lambda \cdot (\frac{\sigma}{\mu})^2 = \lambda \cdot \left( \frac{\sqrt{\frac{1}{m} \sum_{i=1}^{m} (p_{i} - \frac{1}{m} \sum_{i=1}^{m} p_{i})^2}}{\frac{1}{m} \sum_{i=1}^{m} p_{i}} \right)^2
$$

其中：

- $p_{i} = \frac{1}{n} \sum_{i=1}^{n} g_{i}(x)$ （专家 $i$ 的平均路由概率），$n$ 是批次中样本的数量，$g_{i}(x)$ 表示路由网络样本 $x$ 分配给专家 $i$ 的概率（经 softmax 归一化后的值）
- $m$ 是专家数量
- $\lambda$ 是控制损失权重的超参数

负载均衡的辅助损失函数还有诸多其他变体，如平方和形式、KL散度形式的损失、方差形式的损失。

变异系数：[wikipedia-Coefficient of variation](https://en.wikipedia.org/wiki/Coefficient_of_variation) \
推荐观看：[知乎-MoE模型技术细节](https://zhuanlan.zhihu.com/p/25326919151)

#### 代码实现如下：

In [ ]:
class CoefficientOfVariationLoss(nn.Module):
    def __init__(self, lambda_weight=1.0, eps=1e-7):
        """
        基于变异系数（CV）的损失函数，用于MoE模型的负载均衡。

        :param lambda_weight: 损失权重系数，默认1.0
        :param eps: 数值稳定性参数，防止除以零，默认1e-7
        """
        super().__init__()
        self.lambda_weight = lambda_weight
        self.eps = eps

    def forward(self, router_logits):

        # 传入的 router_logits 维度为 (sample_size, expert_num)
        router_probs = F.softmax(router_logits, dim=1)

        # 计算每个专家的平均激活概率，维度为 (expert_num)
        p_i = torch.mean(router_probs, dim=0)

        # 计算均值和标准差（总体标准差，unbiased=False）
        mu = torch.mean(p_i)
        sigma = torch.std(p_i, unbiased=False)  # 使用总体标准差（除以n，而非n-1）

        # 计算变异系数（CV）的平方
        cv_squared = (sigma / (mu + self.eps)) ** 2

        # 加权损失
        loss = self.lambda_weight * cv_squared
        return loss

### 2. 路由的代码实现

注意：这里路由选择的是**处理单个隐状态（这里的隐状态可以认为是一个嵌入了位置信息、语义信息及上下文信息的 token）对应的专家，并且单个隐状态的信息是独立的**，所以对传入的 `x` 而言，路由并不关注其组织结构，比如对于 `(batch, seq_len, dim_emb)` 这样的结构，我们可以将其视作对 `batch * seq_len` 个维度为 `dim_emb` 的隐状态进行处理，故而我们可以忽略 `(batch, seq_len)` 这样的组织结构，并将其直接视作一个 `sample_size` 进行处理，即你可以认为 `sample_size = (batch, seq_len)` 或者 `sample_size = batch * seq_len`。

In [ ]:
class MoERouter(nn.Module):
    def __init__(self, routed_expert_num, dim_emb, top_k = None):
        """
        MoE Router

        :param routed_expert_num: 路由的专家数量
        :param dim_emb: 嵌入维度
        :param top_k: 选取专家的数量，如果为 `None` 则返回所有的专家
        """
        super().__init__()
        self.routed_expert_num = routed_expert_num
        self.dim_emb = dim_emb
        self.top_k = top_k

        self.fnn = nn.Linear(dim_emb, routed_expert_num, bias=False)

    def forward(self, x):

        # 计算专家概率分布
        # logits 和 props 的维度均为 (sample_size, routed_expert_num)
        logits = self.fnn(x)
        props = F.softmax(logits, dim = -1)

        if self.top_k is not None and self.top_k > 0:
            # 根据专家概率大小，取排名前 top_k 个专家的概率和对应的索引
            # expert_probs 是专家对应的概率，维度为 (sample_size ,top_k)
            # expert_idxes 是专家对应的索引，维度为 (sample_size ,top_k)
            expert_probs, expert_idxes = torch.topk(logits, self.top_k)

            # 专家概率归一化
            expert_probs = expert_probs / expert_probs.sum(-1, keepdim = True)

            # 这里防止类型发生改变
            expert_probs = expert_probs.to(x.dtype)

            return logits, expert_idxes, expert_probs
        else:
            return logits, props

#### 路由实现测试

In [ ]:
# 配置
# =====================================
# 输入配置
batch = 3
seq_len = 5
dim_emb = 64

# 路由配置
routed_expert_num = 12
top_k = 2
# =====================================

moe_router = MoERouter(routed_expert_num, dim_emb, top_k)
x = torch.randn((batch, seq_len, dim_emb))

# 获取专家索引和概率
router_logits, expert_idxes, expert_probs = moe_router(x)

print('expert_idxes: ', expert_idxes)
print('expert_probs: ', expert_probs)

## MoE 实现

---

### 1. Dense MoE

Dense MoE （稠密混合专家模型）是一种特殊的 Mixture of Experts（专家混合）模型架构。Dense MoE 在每次前向传播过程中会激活所有的专家网络，而不是只选择其中的一部分。这种设计使得 Dense MoE 能够充分利用所有专家的计算能力，通常能够提供更高的预测准确性。然而，由于所有专家都被激活，Dense MoE 的计算开销会显著增加，这在处理大规模数据或复杂任务时可能会成为一个瓶颈。其原理图如下：

![Dense MoE Struct](img/Dense-MoE-Struct.svg)

（一般而言，Dense MoE 在大语言模型上的应用较少，本节内容不是重点可以选择性跳过。）

In [ ]:
class DenseMoE(nn.Module):
    def __init__(self, expert_num, dim_emb, dim_hidden):
        """
        Dense MoE

        :param expert_num: 专家数量
        :param dim_emb: 嵌入维度
        :param dim_hidden: 专家层的隐藏维度
        """
        super().__init__()
        self.expert_num = expert_num
        self.dim_emb = dim_emb

        # 创建专家
        self.experts = nn.ModuleList()
        for _ in range(expert_num):
            self.experts.append(Expert(dim_emb, dim_hidden))

        # 创建路由
        self.router = MoERouter(expert_num, dim_emb)

    def forward(self, x):

        batch, seq_len, dim_emb = x.size()

        # 合并 batch 和 seq_len 维度，维度          为 (batch * seq_len, dim_emb)
        # 这样做是为了简化专家的实现逻辑，并提高计算效率。
        x = x.view(-1, dim_emb)

        # 进行路由，获取专家索引和对应的概率分布，维度为 (batch * seq_len, expert_num)
        router_logits, expert_weights = self.router(x)

        # 使用对应的专家
        expert_outputs = []
        for expert in self.experts:
            # 扩张维度为 (batch * seq_len, 1, dim_emb)
            expert_output = expert(x).unsqueeze(-2)
            expert_outputs.append(expert_output)

        # 输出维度为 (batch * seq_len, expert_num, dim_emb)
        expert_outputs = torch.cat(expert_outputs, dim = -2)

        # 扩张专家权重维度为 (batch * seq_len, expert_num, 1)
        expert_weights = expert_weights.unsqueeze(-1)

        # 专家输出乘上权重，维度为 (batch * seq_len, expert_num, dim_emb)
        output = expert_weights * expert_outputs

        # 对 expert_num 所在的维度进行求和，输出维度为 (batch * seq_len, dim_emb)
        output = output.sum(dim = -2)

        # 调整维度为 (batch, seq_len, dim_emb)
        output = output.view(batch, seq_len, dim_emb)
        return router_logits, output

有关 37 行代码至 44 行的代码，其数学结果等价于下面的代码：

```python
# 扩张专家权重维度为 (batch * seq_len, 1, expert_num)
expert_weights = expert_weights.unsqueeze(-2)

# 专家输出使用矩阵乘法与其权重相乘，维度为 (batch * seq_len, 1, dim_emb)
output = torch.matmul(expert_weights, expert_outputs)

# 去除额外的维度，维度为 (batch * seq_len, dim_emb)
output = output.squeeze(-2)
```

虽然这样写没有上面给出的代码直观，但是一般情况下矩阵乘法可以利用高度优化的库，减少内存占用和计算时间，即矩阵乘法会比按元素相乘然后求和更高效。

#### Dense MoE 实现测试

In [ ]:
# 配置
# =====================================
# 输入配置
batch = 3
seq_len = 5
dim_emb = 64

# MoE 配置
expert_num = 12
dim_hidden = dim_emb * 4 // expert_num
# =====================================

dense_moe = DenseMoE(expert_num, dim_emb, dim_hidden)
x = torch.randn((batch, seq_len, dim_emb))

router_logits, output = dense_moe(x)
print('output size: ', output.size())
print('output: ', output)

### 2. Sparse MoE

Sparse MoE（稀疏混合专家模型）是一种高效的神经网络架构，其核心思想是为每个输入 token 选择 top-k 个专家网络进行处理，并将这些专家的输出加权组合以生成最终的隐藏状态。在 Sparse MoE 中，路由器（通常是一个可训练的前馈网络）会根据输入 token 的特征，决定将其发送给哪些专家。其原理图如下：

![Sparse MoE Struct](img/Sparse-MoE-Struct.svg)

下面是 Sparse MoE 的未优化版的代码实现：

In [ ]:
class SimpleSparseMoE(nn.Module):
    def __init__(self, expert_num, dim_emb, dim_hidden, top_k):
        super().__init__()
        self.expert_num = expert_num
        self.dim_emb = dim_emb
        self.dim_hidden = dim_hidden
        self.top_k = top_k

        self.experts = nn.ModuleList()
        for _ in range(expert_num):
            self.experts.append(Expert(dim_emb, dim_hidden))

        self.router = MoERouter(expert_num, dim_emb, top_k)

    def forward(self, x):

        batch, seq_len, dim_emb = x.size()

        # 合并 batch 和 seq_len 维度，维度为 (batch * seq_len, dim_emb)
        # 这样做是为了简化专家的实现逻辑，并提高计算效率。
        x = x.view( -1 , dim_emb)

        # 获取专家索引和对应的概率以及专家掩码
        # expert_idxes , expert_probs 的维度为 (batch * seq_len, top_k)
        router_logits, expert_idxes, expert_probs = self.router(x)

        final_outputs = []
        # 对每一个隐状态进行处理
        for hidden_state_idx in range(batch * seq_len):

            # 获取当前隐状态选择的专家索引，维度为 (top_k)
            selected_expert_idxes = expert_idxes[hidden_state_idx]
            # 获取当前隐状态选择的专家概率，维度为 (top_k)
            selected_expert_probs = expert_probs[hidden_state_idx]

            expert_outputs = []
            for idx in selected_expert_idxes:
                # 获取选中的专家
                expert = self.experts[idx]

                # 专家的输出为，扩展维度为 (1, dim_emb)
                expert_output = expert(x[hidden_state_idx]).unsqueeze(-2)
                expert_outputs.append(expert_output)

            # expert_outputs 维度为 (top_k,dim_emb)
            expert_outputs = torch.cat(expert_outputs, dim = -2)

            # 令专家权重为 (1,top_k)
            expert_weights = selected_expert_probs.unsqueeze(-2)

            # 权重叉乘专家输出，输出为 (1, dim_emb)
            expert_outputs = torch.matmul(expert_weights, expert_outputs)
            final_outputs.append(expert_outputs)

        # 合并 final_outputs 维度为 (batch * seq_len, dim_emb)
        final_outputs = torch.cat(final_outputs, dim = -2)

        # 将最终结果转换为 (batch, seq_len, dim_emb)
        output = final_outputs.view(batch, seq_len, dim_emb)

        return router_logits, output

#### 优化版本实现

上面的 `SimpleSparseMoE` 是为了将 MoE 计算流程表述的更清晰，但因为其有多层循环，内存利用率和计算效率都很低，其并不适用于实际场合，所以下面给出其优化版本的实现。下面优化中将会使用到 `torch.where(condition)` 函数，下面介绍一下我们会用到的一种用法：

如果给定布尔张量 `x` 直接使用 `troch.where(x)` ，那么返回的结果有两个向量，第一个返回的是按顺序出现的行为真的索引，第二个返回的是按顺序出现的列为真的索引，如下代码所示：

In [ ]:
# 构建一个专家选择掩码
x = F.one_hot(
    torch.tensor([2,1,3]),
    5
)
print('x: \n', x)

rows, cols = torch.where(x)
print('rows: ', rows)
print('cols: ', cols)

其返回结果为 `tensor([0, 1, 2])` 和 `tensor([2, 1, 3])` ，即表示张量 `x` 的 `(0,2)` 、`(1,1)`、`(2,3)` 位置为真。在下面的实现代码中，我们会将其返回的列用做筛选使用到对应专家的隐状态，返回的行用作确定其对应的  top 顺序（用于确定其对应的权重）。

In [ ]:
class SparseMoE(nn.Module):
    def __init__(self, expert_num, dim_emb, dim_hidden, top_k):
        super().__init__()
        self.expert_num = expert_num
        self.dim_emb = dim_emb
        self.dim_hidden = dim_hidden
        self.top_k = top_k

        self.experts = nn.ModuleList()
        for _ in range(expert_num):
            self.experts.append(Expert(dim_emb, dim_hidden))

        self.router = MoERouter(expert_num, dim_emb, top_k)

    def forward(self, x):

        #===========
        # 优化输入结构
        #===========

        batch, seq_len, dim_emb = x.size()

        # 合并 batch 和 seq_len 维度，维度为 (batch * seq_len, dim_emb)
        # 这样做是为了简化专家的实现逻辑，并提高计算效率。
        hidden_states  = x.view( -1 , dim_emb)

        #===========
        # 路由分配专家
        #===========

        # 获取专家索引和对应的概率以及专家掩码
        # expert_idxes , expert_probs 的维度为 (batch * seq_len, top_k)
        router_logits, expert_idxes, expert_probs = self.router(hidden_states)

        #===========
        # 计算专家输出
        #===========

        # 使用独热编码生成专家掩码，维度为 (batch * seq_len, top_k, expert_num)
        expert_mask = F.one_hot(
            expert_idxes,
            num_classes=self.expert_num,
        )
        # 为方便后续处理，改变专家掩码的维度，使其变为 (expert_num, batch * seq_len, top_k)
        expert_mask = expert_mask.permute(2, 0, 1)

        # 创建一个零张量，为后续累加做准备
        sum_expert_outputs = torch.zeros_like(hidden_states)

        for expert_idx in range(self.expert_num):
            expert = self.experts[expert_idx]

            # hidden_state_idxes 为使用到当前专家的隐状态的索引
            # expert_top_orders 为专家在选择的 top_k 中的排名顺序
            hidden_state_idxes, expert_top_orders = torch.where(expert_mask[expert_idx])

            # 过滤出使用当前专家的隐状态，维度为 (filtered_hidden_states_size, dim_emb)
            filtered_hidden_states = hidden_states[hidden_state_idxes, :]

            # 过滤出隐状态使用当前专家时对应的权重，维度为 (filtered_hidden_states_size)
            filtered_expert_weights = expert_probs[hidden_state_idxes, expert_top_orders]

            # 使用专家处理隐状态并乘上相应的权重，输出维度为 (filtered_hidden_states_size, dim_emb)
            expert_outputs = expert(filtered_hidden_states) * (filtered_expert_weights.unsqueeze(-1))

            # 累加专家输出结果
            # 第一个参数 0 表示沿着隐状态所在的维度进行累加
            # 第二个参数表示要在该维度下的哪些索引进行累加
            # 第三个参数是需要累加的值
            if self.training:
                # 训练模式下使用非原地操作的 index_add ，确保每次累加生成新的张量并保留完整的计算路径。
                sum_expert_outputs = sum_expert_outputs.index_add(0, hidden_state_idxes, expert_outputs.to(hidden_states.dtype))
            else:
                # 非训练模式下使用原地操作的 index_add_ ，加快计算速度
                sum_expert_outputs.index_add_(0, hidden_state_idxes, expert_outputs.to(hidden_states.dtype))

        # 将最终结果转换为 (batch, seq_len, dim_emb)
        output = sum_expert_outputs.reshape(batch, seq_len, self.dim_emb)

        return router_logits, output

#### 优化版本的 Sparse MoE 实现测试

In [ ]:
# 配置
# =====================================
# 输入配置
batch = 3
seq_len = 5
dim_emb = 32

# MoE 配置
expert_num = 12
top_k = 2
dim_hidden = dim_emb * 4 // expert_num
# =====================================

sparse_moe = SparseMoE(expert_num, dim_emb, dim_hidden, top_k)
x = torch.randn((batch, seq_len, dim_emb))

router_logits, output = sparse_moe(x)
print('output size: ', output.size())
print('output: ', output)